In [1]:
#install nfl_data_py package
#!pip install nfl_data_py


In [2]:
import nfl_data_py as nfl


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from tqdm import tqdm

import nfl_data_py as nfl

%matplotlib inline

C:\Users\src15\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
C:\Users\src15\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\src15\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [4]:
from sklearn.metrics import mean_absolute_error

In [5]:
#Calculate the market-implied team total, derived the betting lines

def total_finder(home_or_away,home_total,away_total):
    if home_or_away == 'home':
        total = home_total
    else:
        total = away_total 
    return total

In [6]:
def last_name(full):
  if full == 'DanielThomas':
    return 'Thomas'
  elif full == 'JulioJones':
    return 'Jones'
  elif '.' not in full:
    print(full)
    return full
  else:
    return full.split(".",1)[1]

In [7]:
def team_fixer(team):
    if team == 'SD':
        return 'LAC'
    elif team == 'OAK':
        return 'LV'
    elif team == 'STL':
        return 'LA'
    else:
        return team

In [8]:
def snap_name_changer(player):
    if 'Jr.' in player:
        return player.split(' Jr.')[0]
        #return 'D.J. Davis'
    elif 'II' in player:
        return player.split(' II')[0]
    else:
        return player

In [9]:
def position_handler(pos):
  if pos in ['WR','RB','TE','FB']:
    return pos
  elif pos == 'RB/W':
    return 'RB'
  elif pos == 'WR/R':
    return 'WR'
  elif pos == 'HB':
    return 'RB'
  else:
    return 'other'

In [10]:
import joblib
import pickle


play_by_play_model = pickle.load(open('play_by_play.pkl', 'rb'))


In [11]:
in_season_model = pickle.load(open('beta_model.pkl', 'rb'))

C:\Users\src15\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DummyRegressor from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\src15\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\src15\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator GradientBoostingRegressor from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [12]:
linear_model = pickle.load(open('linear_model.pkl', 'rb'))

In [13]:
from tqdm import tqdm
import tqdm.notebook as tq
import warnings
warnings.filterwarnings('ignore')

# Apply Model

In [14]:
#load in data for NFL pass attempts

szn_receivers=[]
target_list=[]
  
df_list = []
weekly_list = []
target_week = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
week_list1= []


all_throws = []
n_szn = []
plus_1_szn = []

for YEAR in tqdm(range(2022,2023)):
    print(YEAR)
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]

    data['home_implied_total'] = abs(data['total_line']/2 + data['spread_line']/2)
    data['away_implied_total'] = abs(data['total_line']/2 - data['spread_line']/2)
    implied_list = []
    for num in data.index:
        home_number = data.loc[num]['home_implied_total']
        away_number = data.loc[num]['away_implied_total']
        has_ball = data.loc[num]['posteam_type']
        number = total_finder(has_ball,home_number,away_number)
        implied_list.append(number)

    data['implied_posteam_total'] = implied_list
    throws = data[data['air_yards'].isna()==False]
    df = throws[['receiver_player_name','week','posteam','game_id','complete_pass','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','pass_touchdown','down','pass_location','season','home_implied_total','away_implied_total','posteam_type','cp','xyac_mean_yardage','qb_hit','yards_after_catch']]

    current_szn_df = df.dropna(subset=['receiver_player_name'])

  #Create column for throws to the middle (1 for throws to middle, 0 for throws elsewhere)                               
    current_szn_df['middle'] = (current_szn_df['pass_location']=='middle')*1 
    current_szn_df['goal_to_go'] = (current_szn_df['yardline_100']<=10)*1 

    current_szn_df = current_szn_df[['receiver_player_name','week','posteam','yards_gained','down','yardline_100','ydstogo','implied_posteam_total','air_yards','middle','qb_hit','complete_pass','yards_after_catch','cp','xyac_mean_yardage','goal_to_go']].set_index(['receiver_player_name','posteam'])
    current_szn_df['year'] = YEAR
  #apply model to the season's stats
    current_x = current_szn_df.drop(['yards_gained','year','week','goal_to_go','complete_pass','yards_after_catch','cp','xyac_mean_yardage'],axis=1)
    current_szn_df['xYards']=play_by_play_model.predict(current_x)
    current_szn_df = current_szn_df.reset_index()
  
    
    yac_df = current_szn_df[current_szn_df['complete_pass']==1]
    weekly_yac = yac_df.groupby(['receiver_player_name','posteam','week']).agg({'complete_pass':'sum','yards_after_catch':'sum','xyac_mean_yardage':'sum'})
    
    
    
    weekly_pass_offense=current_szn_df.groupby(['posteam','week']).agg({'week':'count','air_yards':'sum'})
    weekly_pass_offense.rename(columns={'week':'team_attempts','air_yards':'team_air_yards'},inplace=True)
    team_attempts=weekly_pass_offense.reset_index()[['posteam','week','team_attempts','team_air_yards']]
    team_attempts.set_index(['posteam','week'],inplace=True)
    weekly_rec_group = current_szn_df.groupby(['receiver_player_name','posteam','week']).agg({'week':'max','down':'count','complete_pass':'sum','yards_gained':'sum','air_yards':'sum','xYards':'sum','cp':'sum','implied_posteam_total':'max','goal_to_go':'sum','qb_hit':'sum','middle':'sum','goal_to_go':'sum'})
    weekly_rec_group.rename(columns={'week':'week#','down':'targets'},inplace=True)
    weekly_rec_group = weekly_rec_group.merge(weekly_yac[['yards_after_catch','xyac_mean_yardage']],right_index=True,left_index=True)
    
    
    feature_receivers = weekly_rec_group.reset_index().set_index(['posteam','week']).merge(team_attempts,right_index=True,left_index=True)
    feature_receivers = feature_receivers.reset_index().set_index(['receiver_player_name','posteam','week'])
    feature_receivers['szn'] = YEAR
    
    week_df1 = feature_receivers.copy()
    week_df = week_df1.copy()
    week_df = week_df.reset_index().set_index(['receiver_player_name','posteam'])
    feature_df = feature_receivers.copy()
    
    for target in tqdm(target_week):
        feature_receivers = feature_df[feature_df['week#']<target]
    
        player_list = []

        for x in feature_receivers.reset_index().set_index(['receiver_player_name','posteam']).index:
            player_list.append(x)
    
    
        players = list(set(player_list))

        print(target)
        for player in players:
            feature_receivers['xYards_median'] = feature_receivers['xYards']
            weekly_player = pd.DataFrame(feature_receivers.loc[player].sort_values('week#',ascending=False).head(6).agg({'yards_gained':'sum','targets':'sum','complete_pass':'sum','week#':'count','szn':'max','air_yards':'sum','xYards':'sum','cp':'sum','goal_to_go':'sum','qb_hit':'sum','middle':'sum','goal_to_go':'sum','team_attempts':'sum','team_air_yards':'sum','yards_after_catch':'sum','xyac_mean_yardage':'sum','xYards_median':'median'})).transpose()
            weekly_player['player'] = player[0]
            weekly_player['team'] = player[1]
            weekly_player['target_week'] = target
            weekly_player.set_index(['player','team','target_week','szn'],inplace=True)
            
          
            df_list.append(weekly_player)
        
        week_df = week_df1.copy()
        
        week_df = week_df[week_df['week#']==target]
        week_df = week_df.reset_index().rename(columns={'week':'target_week','receiver_player_name':'player','posteam':'team'})
        week_df = week_df.set_index(['player','team','target_week','szn'])

        week_list1.append(week_df)
        
        
        
        
        
weekly_receivers = pd.concat(df_list)
weekly_targets = pd.concat(week_list1)
        
        #weekly_receivers['aDOT']=weekly_receivers['air_yards']/weekly_receivers['targets']
        #weekly_receivers['AY_share']=round(weekly_receivers['air_yards']/weekly_receivers['team_air_yards'],3)
        #weekly_receivers['target_share']=round(weekly_receivers['targets']/weekly_receivers['team_attempts'],3)
        #weekly_receivers['yac/rec']=round(weekly_receivers['yards_after_catch']/weekly_receivers['complete_pass'],3)
        #weekly_receivers['xyac/rec']=round(weekly_receivers['xyac_mean_yardage']/weekly_receivers['complete_pass'],3)


        
        

  0%|          | 0/1 [00:00<?, ?it/s]

2022



  0%|          | 0/18 [00:00<?, ?it/s]

1
2



 11%|█         | 2/18 [00:02<00:20,  1.31s/it]

...


 89%|████████▉ | 16/18 [01:09<00:10,  5.37s/it]

17



 94%|█████████▍| 17/18 [01:15<00:05,  5.64s/it]

18



100%|██████████| 1/1 [02:05<00:00, 125.10s/it]


In [15]:
feature_df['target_week'] = feature_df['week#']+1

feature_df.drop(('A.St. Brown','DET',7),inplace=True)

In [16]:
feature_df.loc['T.Lockett']

week#  targets  complete_pass  yards_gained  air_yards  \
posteam week                                                           
SEA     1         1        4            3.0          28.0       29.0   
        2         2       11            9.0         107.0      118.0   
        3         3       11            9.0          76.0      123.0   
        4         4        8            6.0          91.0      122.0   
        5         5        6            5.0         104.0      135.0   
        6         6        5            2.0          17.0       62.0   
        7         7        8            7.0          45.0       43.0   
        8         8        8            5.0          63.0       84.0   
        9         9        5            5.0          67.0       24.0   
        10       10        5            3.0          42.0       53.0   
        12       12        7            3.0          68.0       90.0   
        13       13       12            9.0         128.0       89.0   
        14       14        9            5.0          60.0       97.0   
        15       15        9            7.0          68.0       67.0   

                 xYards        cp  implied_posteam_total  goal_to_go  qb_hit  \
posteam week                                                                   
SEA     1     29.970425  2.799239                  19.00           0     0.0   
        2     80.492153  6.903809                  15.75           0     2.0   
        3     84.512757  6.664902                  22.00           0     1.0   
        4     65.110391  4.626942                  22.75           1     2.0   
        5     51.207771  3.103960                  20.00           0     2.0   
        6     39.778139  3.077915                  23.75           0     0.0   
        7     52.408489  5.625344                  23.00           0     1.0   
        8     51.552411  4.539161                  23.75           0     2.0   
        9     31.774134  3.406899                  24.00           1     1.0   
        10    37.522324  3.040777                  21.25           0     0.0   
        12    64.875953  4.536230                  25.75           0     1.0   
        13    89.451189  8.285791                  24.00           1     1.0   
        14    72.222910  5.002261                  24.00           1     1.0   
        15    66.285869  6.362094                  20.00           0     0.0   

              middle  yards_after_catch  xyac_mean_yardage  team_attempts  \
posteam week                                                                
SEA     1          1                5.0          15.557277             27   
        2          3               22.0          36.641333             30   
        3          0                3.0          34.271528             44   
        4          2               11.0          22.844684             29   
        5          2                5.0          10.438336             24   
        6          2                8.0           7.939587             30   
        7          1                9.0          34.396273             26   
        8          0               28.0          27.092477             33   
        9          1               43.0          27.223590             32   
        10         1                9.0           8.453819             33   
        12         5               24.0           9.173747             37   
        13         2               68.0          43.275493             37   
        14         0                1.0          18.435113             36   
        15         1               21.0          31.932819             41   

              team_air_yards   szn  target_week  
posteam week                                     
SEA     1              135.0  2022            2  
        2              170.0  2022            3  
        3              412.0  2022            4  
        4              319.0  2022            5  
        5              300.0  2022            6

In [17]:
    YEAR=2022
    
    
    snap_df = nfl.import_snap_counts([YEAR])
    snap_df['team'] = snap_df['team'].apply(lambda x: team_fixer(x))
    snap_df = snap_df[snap_df['game_type']=='REG']
    snap_df = snap_df[['pfr_player_id','player','offense_snaps','offense_pct','week','team','game_id','position']]
    snap_df['player_copy'] = snap_df['player']
    snap_df.rename(columns={'player_copy':'full_name'},inplace=True)
    snap_df['full_name'] = snap_df['full_name'].apply(lambda x: snap_name_changer(x))
    
    
    snap_df['week#'] = snap_df['week']
    #snap_df = snap_df.set_index(['receiver_player_id','team','week'])
    snap_df = snap_df[snap_df['offense_snaps']>0]

    snap_df['last_name'] = snap_df['full_name'].apply(lambda x: x.split(" ",1)[1])
    snap_df['identifier'] = (snap_df['full_name']).str[0]+"."+snap_df['last_name']+"_"+snap_df['team']+"_"+(snap_df['week'].astype(str))
    snap_df = snap_df[['full_name','identifier','offense_snaps','position','team']]




In [18]:
snap_df = snap_df[snap_df['identifier']!='A.St. Brown_DET_7']

snap_df = snap_df[snap_df['identifier']!='D.Metcalf_SEA_7']

In [19]:
snap_df = snap_df[snap_df['identifier']!='M.Andrews_BAL_8']


In [20]:
player_df_list = []



for player_id in tq.tqdm(feature_df.index):

  player = player_id[0:2]
  player_df = feature_df.loc[player]
  week_df=pd.DataFrame()
  week_df['week#']=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
  player_df = player_df.merge(week_df,how='right')
  player_df['receiver_player_name'] = player[0]
  
  player_df['posteam'] = player[1]
  player_df['last'] = player_df['receiver_player_name'].apply(lambda x: last_name(x))
  player_df['identifier'] = (player_df['receiver_player_name']).str[0]+"."+player_df['last']+"_"+player_df['posteam']+"_"+player_df['week#'].astype(str)
  player_df = player_df.merge(snap_df,on='identifier')



  player_df.fillna(0,inplace=True)

  player_df['xYards_median'] = player_df['xYards'].rolling(6,min_periods=6).median()
  player_df['xYards/game'] = player_df['xYards'].rolling(6,min_periods=6).mean()

  player_df['xYards_sum'] = player_df['xYards'].rolling(6,min_periods=6).sum()
  player_df['yards_sum'] = player_df['yards_gained'].rolling(6,min_periods=6).sum()
  player_df['yards/game'] = player_df['yards_gained'].rolling(6,min_periods=6).mean()
  player_df['games'] = player_df['week#'].rolling(6,min_periods=1).count()
  player_df.rename(columns={'week#':'week'},inplace=True)
  target_df1 = pd.DataFrame()


  player_df['target_week'] = player_df['week']+1

  target_df1['target_week']=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

  player_df = player_df.merge(target_df1,how='right').sort_values('target_week')




  player_df = player_df.fillna(method='ffill')

  player_df_list.append(player_df)




  0%|          | 0/3174 [00:00<?, ?it/s]

In [21]:
trailing_weeks = pd.concat(player_df_list)


In [22]:
new_df = trailing_weeks[['receiver_player_name','full_name','xYards','yards_gained','week','posteam']].drop_duplicates()



In [23]:
new_df = new_df[new_df['receiver_player_name'].isna()==False]

## 5. Actual Results

### Week 8

In [24]:
week=8

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6


trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])

#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])


trailing_weeks

AttributeError: 'GradientBoostingRegressor' object has no attribute 'n_features_'

In [ ]:
yardage_trailing.reset_index().rename(columns={'index':'player'})


In [ ]:
projections = trailing_weeks.sort_values('pYards',ascending=False)

#[['player','team','pYards','Yards/game','games','target_share_x','aDOT','xYards/game','team_attempts_x']]

In [ ]:
df=pd.read_csv('betting_lines/nfl-best-bets.csv')
df = df[df['team']!='CLV']

mnf=pd.read_csv('betting_lines/nfl-best-bets (1).csv')

In [ ]:
df = df.append(mnf)

In [ ]:
projections.reset_index(inplace=True)

#projections.rename(columns={'index':'player'},inplace=True)

In [ ]:
names_to_change = ['M.Jones Jr','E.St','I.Smith Jr','A.St','T.Marshall Jr','A.Robinson II','M.Pittman Jr','K.Walker III','M.Gordon III']

def name_changer(name):
    if name in names_to_change:
        if name == 'M.Jones Jr':
            return 'M.Jones'
        if name == 'E.St':
            return 'E.St. Brown'
        if name == 'I.Smith Jr':
            return 'I.Smith'
        if name == 'A.St':
            return 'A.St. Brown'
        if name == 'T.Marshall Jr':
            return 'T.Marshall'
        if name == 'A.Robinson II':
            return 'Al.Robinson'
        if name == 'M.Pittman Jr':
            return 'M.Pittman'
        if name == 'K.Walker III':
            return 'K.Walker'
        if name == 'M.Gordon III':
            return 'M.Gordon'
    if name not in names_to_change:
        return name

In [ ]:
odds = df[['player','team','line']]

def team_changer(name):
    if name == 'HST':
        return 'HOU'
    elif name == 'CLV':
        return 'CLE'
    elif name == 'ARZ':
        return 'ARI'
    elif name == 'BLT':
        return 'BAL'
    else:
        return name

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))



odds


In [ ]:
odds

In [ ]:
projections

In [ ]:
projections

In [ ]:
table = odds.merge(projections,on=['player','team'])

table = table.round(2)

table['diff'] = table['pYards'] - table['line']




table['abs_diff'] = abs(table['diff'])

In [ ]:
table.sort_values('pYards',ascending=False).head(50)

In [ ]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]

  

In [ ]:
#table.shape[0]


table = table.drop_duplicates()

In [ ]:
table['pYards'].corr(table['line'])


In [ ]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==8]



In [ ]:
weekly_Rec = results.groupby('receiver_player_name').agg({'yards_gained':'sum','pass':'count','complete_pass':'sum'}).reset_index().rename(columns={'receiver_player_name':'player'})

In [ ]:
weekly_Rec

In [ ]:
table

In [ ]:
#table.drop(columns={'yards_gained'},inplace=True)

In [ ]:
smash_table = table[table['abs_diff']>=10]

results = table.merge(weekly_Rec,on='player')


In [ ]:
results=results.dropna()



In [ ]:
results['Over_Picked'] =results['pYards']>results['line']

In [ ]:
results['Over_Result'] = results['yards_gained'] > results['line']

results['Correct'] = results['Over_Picked'] == results['Over_Result']

In [ ]:
w8_results = results[['player','team','line','pYards','yards_gained','Over_Picked','Over_Result','Correct']]

w8_results['week'] = 8

w8_results = w8_results[['player','team','line','pYards','yards_gained','Over_Picked','Over_Result','Correct']]


In [ ]:
results['yards_gained'].corr(results['pYards'])

In [ ]:
#0.5860590594191387

In [ ]:
import math
MSE = np.square(np.subtract(results['yards_gained'],results['pYards'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error (players w/ listed odds):\n")
print(RMSE)

print('-------------')

weekly_all  = weekly_Rec.merge(projections)[['player','pYards','yards_gained']]

MSE = np.square(np.subtract(weekly_all['yards_gained'],weekly_all['pYards'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error (all players):\n")
print(RMSE)

In [ ]:
#30.880608939504032
#-------------

#30.24493362852292

In [ ]:
#overall
results['Correct'].value_counts()

In [ ]:
smash_results = results[results['abs_diff']>=10]

smash_results['Correct'].value_counts()

In [ ]:
smash_results[['player','line','xYards/game','pYards','abs_diff','yards_gained','Over_Picked','Over_Result','Correct']].sort_values('abs_diff',ascending=False)

In [ ]:
picked_overs = results[results['Over_Picked']==True]

picked_overs['Correct'].value_counts()

In [ ]:
picked_unders = results[results['Over_Picked']==False]

picked_unders['Correct'].value_counts()

In [ ]:
results['Over_Result'].value_counts()

In [ ]:
smash_overs = smash_results[smash_results['Over_Picked']==True]


smash_overs['Correct'].value_counts()

In [ ]:
smash_unders = smash_results[smash_results['Over_Picked']==False]


smash_unders['Correct'].value_counts()

In [ ]:
table[table['team']=='CLE']

### Week 9

In [ ]:
week=9

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

trailing_weeks

In [ ]:
trailing_weeks.drop_duplicates().shape[0]

In [ ]:
trailing_weeks

In [ ]:
w9_table = trailing_weeks.copy()


In [ ]:
#w9_table = w9_table.reset_index().rename(columns={'index':'player'})

In [ ]:
w9_table.set_index(['player','team'],inplace=True)

In [ ]:
w9_odds = pd.read_csv('betting_lines/nfl-best-bets (3).csv').round(1)[['player','team','line']]
w9_odds = w9_odds[w9_odds['team']!='BLT']
w9_odds = w9_odds[w9_odds['team']!='NO']


w9_odds = w9_odds.append(pd.read_csv('betting_lines/nfl-best-bets (4).csv').round(1)[['player','team','line']])



odds = w9_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


In [ ]:
odds.set_index(['player','team'],inplace=True)

In [ ]:

w9_table = w9_table.merge(odds,right_index=True,left_index=True)
w9_table = w9_table.reset_index()

w9_table['abs. diff'] = abs(w9_table['pYards'] - w9_table['line'])

w9_table = w9_table.round(1)

w9_table['Over_Picked'] =w9_table['pYards']>w9_table ['line']

#w9_table.sort_values('abs. diff',ascending=False).head(50)


In [ ]:
w9_table[w9_table['team']=='BAL']

In [ ]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==9]


In [ ]:
results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w9_table = w9_table.merge(results)

w9_table['Over_Result'] = w9_table['yards_gained']>w9_table['line']

w9_table['Win'] = (w9_table['Over_Result'] == w9_table['Over_Picked'])*1



In [ ]:

import math
MSE = np.square(np.subtract(w9_table['pYards'],w9_table['yards_gained'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:\n")
print(RMSE)

import math
MSE = np.square(np.subtract(w9_table['yards_gained'],w9_table['line'])).mean() 
 
print('-----')
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting Line:\n")
print(RMSE)

In [ ]:
w9_table['week'] = 9

In [ ]:
w8_results['Correct'] = w8_results['Correct']*1
w8_results.rename(columns={'Correct':'Win'},inplace=True)

In [ ]:
w8_results[w8_results['team']=='DET']

In [ ]:
w8_results['week']=8

### Week 10

In [ ]:
week=10

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6



trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

trailing_weeks

In [ ]:
w10_table = trailing_weeks.copy()


#w10_table = w10_table.reset_index().rename(columns={'index':'player'})

In [ ]:
w10_table.set_index(['player','team'],inplace=True)

In [ ]:
w10_odds = pd.read_csv('betting_lines/nfl-best-bets (5).csv').round(1)[['player','team','line']]






odds = w10_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

w10_table = w10_table.merge(odds,right_index=True,left_index=True)
w10_table = w10_table.reset_index()

w10_table['abs. diff'] = abs(w10_table['pYards'] - w10_table['line'])

w10_table = w10_table.round(1)

w10_table['Over_Picked'] =w10_table['pYards']>w10_table ['line']

w10_table.sort_values('abs. diff',ascending=True).tail(25)

In [ ]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==10]


In [ ]:
results

In [ ]:
results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w10_table = w10_table.merge(results)

w10_table['Over_Result'] = w10_table['yards_gained']>w10_table['line']

w10_table['Win'] = (w10_table['Over_Result'] == w10_table['Over_Picked'])*1


w10_table['week'] = 10


In [ ]:

import math
MSE = np.square(np.subtract(w10_table['pYards'],w10_table['yards_gained'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:\n")
print(RMSE)

import math
MSE = np.square(np.subtract(w10_table['yards_gained'],w10_table['line'])).mean() 
 
print('-----')
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting Line:\n")
print(RMSE)

In [ ]:
w10_table[w10_table['team']=='IND']

### Week 11

In [ ]:
week=11

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6



trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

trailing_weeks

In [ ]:
w11_table = trailing_weeks.copy()

#w11_table = w11_table.reset_index().rename(columns={'index':'player'})

w11_table.set_index(['player','team'],inplace=True)

w11_odds = pd.read_csv('betting_lines/nfl-best-bets (8).csv').round(1)[['player','team','line']]






odds = w11_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

w11_table = w11_table.merge(odds,right_index=True,left_index=True)
w11_table = w11_table.reset_index()

w11_table['abs. diff'] = abs(w11_table['pYards'] - w11_table['line'])

w11_table = w11_table.round(1)

w11_table['Over_Picked'] =w11_table['pYards']>w11_table ['line']


def o_u(over):
  if over == True:
    return 'Over'
  elif over == False:
    return 'Under'



In [ ]:
tnf = w11_table[(w11_table['team']=='GB')|(w11_table['team']=='TEN')]


In [ ]:
tnf

In [ ]:
w11_table = trailing_weeks.copy()

#w11_table = w11_table.reset_index().rename(columns={'index':'player'})

w11_table.set_index(['player','team'],inplace=True)

w11_odds = pd.read_csv('betting_lines/nfl-best-bets (9).csv').round(1)[['player','team','line']]






odds = w11_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

w11_table = w11_table.merge(odds,right_index=True,left_index=True)
w11_table = w11_table.reset_index()

w11_table['abs. diff'] = abs(w11_table['pYards'] - w11_table['line'])

w11_table = w11_table.round(1)

w11_table['Over_Picked'] =w11_table['pYards']>w11_table ['line']


w11_table = w11_table.append(tnf)



In [ ]:
snf = trailing_weeks.copy()


#snf = snf.reset_index().rename(columns={'index':'player'})

snf.set_index(['player','team'],inplace=True)

snf_odds = pd.read_csv('betting_lines/nfl-best-bets (10).csv').round(1)[['player','team','line']]






odds = snf_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

snf = snf.merge(odds,right_index=True,left_index=True)
snf = snf.reset_index()

snf['abs. diff'] = abs(snf['pYards'] - snf['line'])

snf = snf.round(1)

snf['Over_Picked'] =snf['pYards']>snf ['line']


primetime = snf[(snf['team']=='KC')|(snf['team']=='LAC')]

In [ ]:
w11_table = w11_table.append(primetime)

In [ ]:
mnf = trailing_weeks.copy()


#mnf = mnf.reset_index().rename(columns={'index':'player'})

mnf.set_index(['player','team'],inplace=True)

mnf_odds = pd.read_csv('betting_lines/nfl-best-bets (11).csv').round(1)[['player','team','line']]






odds = mnf_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

mnf = mnf.merge(odds,right_index=True,left_index=True)
mnf = mnf.reset_index()

mnf['abs. diff'] = abs(mnf['pYards'] - mnf['line'])

mnf = mnf.round(1)
mnf['Over_Picked'] =mnf['pYards']>mnf ['line']


mnf = mnf[(mnf['team']=='ARI')|(mnf['team']=='SF')]

In [ ]:
mnf

In [ ]:
w11_table = w11_table.append(mnf)

In [ ]:
w11_table[(w11_table['team']=='ARI')|(w11_table['team']=='SF')]

In [ ]:
w11_table[w11_table['abs. diff']>=10].sort_values('abs. diff',ascending=False)

In [ ]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==11]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w11_table = w11_table.merge(results)

w11_table['Over_Result'] = w11_table['yards_gained']>w11_table['line']

w11_table['Win'] = (w11_table['Over_Result'] == w11_table['Over_Picked'])*1


w11_table['week'] = 11

In [ ]:
w11_table[w11_table['abs. diff']>=10].sort_values('abs. diff',ascending=False)['Win'].mean()

In [ ]:

import math
MSE = np.square(np.subtract(w10_table['pYards'],w10_table['yards_gained'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:\n")
print(RMSE)

import math
MSE = np.square(np.subtract(w10_table['yards_gained'],w10_table['line'])).mean() 
 
print('-----')
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting Line:\n")
print(RMSE)

### Week 12

In [ ]:
week=12

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

trailing_weeks

In [ ]:


w12_table = trailing_weeks.copy()

 
#w12_table = w12_table.reset_index().rename(columns={'index':'player'})
 
w12_table.set_index(['player','team'],inplace=True)
 
w12_odds = pd.read_csv('betting_lines/nfl-best-bets (12).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w12_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w12_table = w12_table.merge(odds,right_index=True,left_index=True)
w12_table = w12_table.reset_index()
 
w12_table['abs. diff'] = abs(w12_table['pYards'] - w12_table['line'])
 
w12_table = w12_table.round(1)
 
w12_table['Over_Picked'] =w12_table['pYards']>w12_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 



In [ ]:
tg = w12_table[w12_table['team']!='CAR'].copy()

In [ ]:


w12_table = trailing_weeks.copy()

 
#w12_table = w12_table.reset_index().rename(columns={'index':'player'})
 
w12_table.set_index(['player','team'],inplace=True)
 
w12_odds = pd.read_csv('betting_lines/nfl-best-bets (13).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w12_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w12_table = w12_table.merge(odds,right_index=True,left_index=True)
w12_table = w12_table.reset_index()
 
w12_table['abs. diff'] = abs(w12_table['pYards'] - w12_table['line'])
 
w12_table = w12_table.round(1)
 
w12_table['Over_Picked'] =w12_table['pYards']>w12_table ['line']

w12_table = w12_table.append(tg)


In [ ]:


mnf12 = trailing_weeks.copy()

 
#mnf12 = mnf12.reset_index().rename(columns={'index':'player'})
 
mnf12.set_index(['player','team'],inplace=True)
 
mnf12_odds = pd.read_csv('betting_lines/nfl-best-bets (14).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = mnf12_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
mnf12 = mnf12.merge(odds,right_index=True,left_index=True)
mnf12 = mnf12.reset_index()
 
mnf12['abs. diff'] = abs(mnf12['pYards'] - mnf12['line'])
 
mnf12 = mnf12.round(1)
 
mnf12['Over_Picked'] =mnf12['pYards']>mnf12 ['line']

w12_table = w12_table.append(mnf12)


In [ ]:
mnf12

In [ ]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==12]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w12_table = w12_table.merge(results)

w12_table['Over_Result'] = w12_table['yards_gained']>w12_table['line']

w12_table['Win'] = (w12_table['Over_Result'] == w12_table['Over_Picked'])*1


w12_table['week'] = 12

### Week 13

In [ ]:
week=13

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])



trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

trailing_weeks

In [ ]:
tnf

In [ ]:
w13_table = trailing_weeks.copy()

 
#w13_table = w13_table.reset_index().rename(columns={'index':'player'})
 
w13_table.set_index(['player','team'],inplace=True)
 
w13_odds = pd.read_csv('betting_lines/nfl-best-bets (20).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w13_table = w13_table.merge(odds,right_index=True,left_index=True)
w13_table = w13_table.reset_index()
 
w13_table['abs. diff'] = abs(w13_table['pYards'] - w13_table['line'])
 
w13_table = w13_table.round(1)
 
w13_table['Over_Picked'] =w13_table['pYards']>w13_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 


#w13_table['mean_diff'] = abs(w13_table['xYards/game']-w13_table['line'])

tnf.append(w13_table[(w13_table['team']!='BUF')&(w13_table['team']!='NE')])

In [ ]:
w13_table = w13_table[w13_table['team']!='IND']

In [ ]:
w13_table = w13_table[w13_table['team']!='DAL']

w13_table = w13_table[w13_table['team']!='NO']

w13_table = w13_table[w13_table['team']!='TB']

In [ ]:
w13 = trailing_weeks.sort_values('pYards',ascending=False)


#w13[(w13['team']=='IND')|(w13['team']=='DAL')]

In [ ]:
snf13 = trailing_weeks.copy()


 
#snf13 = snf13.reset_index().rename(columns={'index':'player'})
 
snf13.set_index(['player','team'],inplace=True)
 
snf13_odds = pd.read_csv('betting_lines/nfl-best-bets (21).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = snf13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
snf13 = snf13.merge(odds,right_index=True,left_index=True)
snf13 = snf13.reset_index()
 
snf13['abs. diff'] = abs(snf13['pYards'] - snf13['line'])
 
snf13 = snf13.round(1)
 
snf13['Over_Picked'] =snf13['pYards']>snf13 ['line']

In [ ]:
mnf13 = trailing_weeks.copy()

 
#mnf13 = mnf13.reset_index().rename(columns={'index':'player'})
 
mnf13.set_index(['player','team'],inplace=True)
 
mnf13_odds = pd.read_csv('betting_lines/nfl-best-bets (23).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = mnf13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
mnf13 = mnf13.merge(odds,right_index=True,left_index=True)
mnf13 = mnf13.reset_index()
 
mnf13['abs. diff'] = abs(mnf13['pYards'] - mnf13['line'])
 
mnf13 = mnf13.round(1)
 
mnf13['Over_Picked'] = mnf13['pYards']>mnf13 ['line']

In [ ]:
#mnf13

In [ ]:
w13_table = w13_table.append(snf13)

w13_table = w13_table.append(mnf13)

In [ ]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==13]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w13_table = w13_table.merge(results)

w13_table['Over_Result'] = w13_table['yards_gained']>w13_table['line']

w13_table['Win'] = (w13_table['Over_Result'] == w13_table['Over_Picked'])*1


w13_table['week'] = 13

In [ ]:
w13_table[w13_table['abs. diff']>=10].sort_values('abs. diff',ascending=False).head(50)

### Week 14

In [ ]:
week=14

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

trailing_weeks

In [ ]:


w14_table = trailing_weeks.copy()

 
#w14_table = w14_table.reset_index().rename(columns={'index':'player'})
 
w14_table.set_index(['player','team'],inplace=True)
 
w14_odds = pd.read_csv('betting_lines/nfl-best-bets (24).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w14_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w14_table = w14_table.merge(odds,on=['player','team'])
w14_table = w14_table.reset_index()
 
w14_table['abs. diff'] = abs(w14_table['pYards'] - w14_table['line'])
 
w14_table = w14_table.round(1)
 
w14_table['Over_Picked'] =w14_table['pYards']>w14_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 
tnf14 = w14_table.copy()



In [ ]:


w14_table = trailing_weeks.copy()

w14_table.set_index(['player','team'],inplace=True)
 
#w14_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
#w14_table.set_index(['player','team'],inplace=True)
 
w14_odds = pd.read_csv('betting_lines/nfl-best-bets (27).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w14_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w14_table = w14_table.merge(odds,on=['player','team'])
w14_table = w14_table.reset_index()
 
w14_table['abs. diff'] = abs(w14_table['pYards'] - w14_table['line'])
 
w14_table = w14_table.round(1)
 
w14_table['Over_Picked'] =w14_table['pYards']>w14_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 
w14_table = tnf14.append(w14_table)

w14_table = w14_table[(w14_table['team']!='LAC') & (w14_table['team']!='MIA')]

w14_table = w14_table[(w14_table['team']!='ARI') & (w14_table['team']!='NE')]

In [ ]:
w14_table

In [ ]:


snf14 = trailing_weeks.copy()

 
 
snf14.set_index(['player','team'],inplace=True)
 
snf14_odds = pd.read_csv('betting_lines/nfl-best-bets (28).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = snf14_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
snf14 = snf14.merge(odds,right_index=True,left_index=True)
snf14 = snf14.reset_index()
 
snf14['abs. diff'] = abs(snf14['pYards'] - snf14['line'])
 
snf14 = snf14.round(1)
 
snf14['Over_Picked'] =snf14['pYards']>snf14 ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 
snf14 = snf14[(snf14['team']=='LAC')|(snf14['team']=='MIA')]

In [ ]:


mnf14 = trailing_weeks.copy()

 
 
mnf14.set_index(['player','team'],inplace=True)
 
mnf14_odds = pd.read_csv('betting_lines/nfl-best-bets (30).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = mnf14_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
mnf14 = mnf14.merge(odds,right_index=True,left_index=True)
mnf14 = mnf14.reset_index()
 
mnf14['abs. diff'] = abs(mnf14['pYards'] - mnf14['line'])
 
mnf14 = mnf14.round(1)
 
mnf14['Over_Picked'] =mnf14['pYards']>mnf14 ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])


mnf14 = mnf14[(mnf14['team']=='NE')|(mnf14['team']=='ARI')]



In [ ]:
mnf14

In [ ]:
w14_table = w14_table.append(snf14).append(mnf)

In [ ]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'
    
    
    

    data.reset_index(drop=True, inplace=True)


    data = data[data['two_point_attempt']==0]

    data['home_implied_total'] = abs(data['total_line']/2 + data['spread_line']/2)
    data['away_implied_total'] = abs(data['total_line']/2 - data['spread_line']/2)
    implied_list = []
    for num in data.index:
        home_number = data.loc[num]['home_implied_total']
        away_number = data.loc[num]['away_implied_total']
        has_ball = data.loc[num]['posteam_type']
        number = total_finder(has_ball,home_number,away_number)
        implied_list.append(number)

    data['implied_posteam_total'] = implied_list
    
    
    
    data = data[data['two_point_attempt']==0]
    
    results = data[data['week']==14]
    results['middle'] = (results['pass_location']=='middle')*1 
    results['goal_to_go'] = (results['yardline_100']<=10)*1
    results = results[['receiver_player_name','posteam','yards_gained','down','yardline_100','ydstogo','implied_posteam_total','air_yards','middle','qb_hit']].set_index(['receiver_player_name','posteam'])
    results = results.dropna()
    
    results['xYards']=play_by_play_model.predict(results.drop(columns={'yards_gained'}))

    
    
    




In [ ]:
results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum','xYards':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained','xYards']]

w14_table = w14_table.merge(results)

w14_table['Over_Result'] = w14_table['yards_gained']>w14_table['line']

w14_table['Win'] = (w14_table['Over_Result'] == w14_table['Over_Picked'])*1


w14_table['week'] = 14

### Week 15

In [ ]:
week=15

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])



In [ ]:
beta_table = trailing_weeks[['player','team','pYards']]

beta_table.rename(columns={'pYards':'beta_yards'},inplace=True)

beta_table.to_csv('wk15_beta.csv')

In [ ]:


weekly_table = trailing_weeks.copy()

weekly_table.set_index(['player','team'],inplace=True)
 
#w14_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
#w14_table.set_index(['player','team'],inplace=True)
 
w15_odds = pd.read_csv('betting_lines/nfl-best-bets (32).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w15_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,on=['player','team'])
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['pYards'] - weekly_table['line'])
 
weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['pYards']>weekly_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 
#weekly_table = tnf14.append(w14_table)

#w14_table = w14_table[(w14_table['team']!='LAC') & (w14_table['team']!='MIA')]

#w14_table = w14_table[(w14_table['team']!='ARI') & (w14_table['team']!='NE')]

In [ ]:
w15_table = weekly_table.copy()

#w15_table.sort_values('abs. diff',ascending=False).head(25)

In [ ]:
beta_table

## Overall Results

In [ ]:
szn_results = w8_results.append(w9_table).append(w10_table).append(w11_table).append(w12_table).append(w13_table).append(w14_table)

szn_results['abs_diff'] = abs(szn_results['pYards']-szn_results['line'])

szn_results['smash'] = szn_results['abs_diff']>=10

smash_df = szn_results[szn_results['smash']==True]


In [ ]:
print('R Values:\n')


print('Yards and projection: {}'.format(round(szn_results['pYards'].corr(szn_results['yards_gained']),3)))
print('Yards and betting line: {}'.format(round(szn_results['line'].corr(szn_results['yards_gained']),3)))
print('Projection and betting line: {}'.format(round(szn_results['pYards'].corr(szn_results['line']),3)))


In [ ]:


print('Overall Record: {}-{} ({}%)'.format(szn_results['Win'].sum(),szn_results.shape[0]-szn_results['Win'].sum(),round(szn_results['Win'].sum()/szn_results.shape[0]*100,1)))

print('Plays Record (10+ yard difference): {}-{} ({}%)'.format(smash_df['Win'].sum(),smash_df.shape[0]-smash_df['Win'].sum(),round(smash_df['Win'].sum()/smash_df.shape[0]*100,1)))

print('\n-----\n')
 
mae = mean_absolute_error(szn_results['pYards'], szn_results['yards_gained'])
    
print("Mean Absolute Error - Projections:")
print('{}\n'.format(round(mae,1)))


mae = mean_absolute_error(szn_results['yards_gained'],szn_results['line'])
 
print("Mean Absolute Error - Betting line:")
print(round(mae,1))

In [ ]:
MSE = np.square(np.subtract(szn_results['yards_gained'],szn_results['pYards'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:")
print('{}\n'.format(round(RMSE,1)))


MSE = np.square(np.subtract(szn_results['yards_gained'],szn_results['line'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting line:")
print(round(RMSE,1))

In [ ]:
print('WEEKLY RESULTS')

print('\nOverall')
for week in szn_results['week'].unique():
    w = szn_results[szn_results['week']==week]['Win'].sum()
    p = szn_results[szn_results['week']==week]['Win'].shape[0]
    l = p - w
    perc = round(w/p*100,1)
    print('Week {}: {}-{} ({}%)'.format(week,w,l,perc))


print('\nPlays')

for week in smash_df['week'].unique():
    w = smash_df[smash_df['week']==week]['Win'].sum()
    p = smash_df[smash_df['week']==week]['Win'].shape[0]
    l = p - w
    perc = round(w/p*100,1)
    print('Week {}: {}-{} ({}%)'.format(week,w,l,perc))

In [ ]:
szn_results['Over_Picked'].value_counts()

In [ ]:
szn_results['Over_Result'].value_counts()

In [ ]:
over_w = szn_results[szn_results['Over_Picked']==1]['Win'].mean()
under_w = szn_results[szn_results['Over_Picked']==0]['Win'].mean()

print('Overs: {}%'.format(round(over_w*100,1)))
print('Under: {}%'.format(round(under_w*100,1)))

In [ ]:
over_w = smash_df[smash_df['Over_Picked']==1]['Win'].mean()
under_w = smash_df[smash_df['Over_Picked']==0]['Win'].mean()

print('Smash Overs: {}%'.format(round(over_w*100,1)))
print('Smash Under: {}%'.format(round(under_w*100,1)))

In [ ]:
szn_results[(szn_results['week']==14)&(szn_results['smash']==True)]

In [ ]:
#szn_results['the_blend'] = (szn_results['xYards_median']+szn_results['pYards'])/2

#szn_results[['pYards','yards_gained','xYards/game','line','the_blend','xYards_median','gb_projection','linear_projection']].corr().sort_values('yards_gained',ascending=False)